In [ ]:
import pandas as pd


# Read data and preprocess
df = pd.read_csv('combined_data_9gauges.csv')
df['DATETIME'] = pd.to_datetime(df['DATETIME'])


# Analyze rainfall events and store in a new DataFrame
events = []
last_event_end = {station: None for station in df['STATION'].unique()}  # Dictionary to store the end time of the last event for each station

for station in df['STATION'].unique():
    station_df = df[df['STATION'] == station]
    event_start = None
    event_end = None
    total_rainfall = 0
    max_value = 0
    event_duration_intervals = 0

    for index, row in station_df.iterrows():
        if row['VALUE'] > 0:
            if event_start is None:
                event_start = row['DATETIME']
                # Calculate intercurrence time in minutes if there was a previous event
                intercurrence = (event_start - last_event_end[station]).total_seconds() / 60 if last_event_end[station] else 0
            event_end = row['DATETIME']
            total_rainfall += row['VALUE']
            max_value = max(max_value, row['VALUE'])
            event_duration_intervals += 1
        else:
            if event_start is not None:
                duration = event_duration_intervals * 10
                # Only append the event if its duration is greater than 0
                if duration > 0:
                    events.append([station, event_start, event_end, total_rainfall, duration, max_value, intercurrence])
                last_event_end[station] = event_end  # Update the end time of the last event for this station
                event_start = None
                total_rainfall = 0
                max_value = 0
                event_duration_intervals = 0

    # Check if the last record is part of an ongoing event
    if event_start is not None:
        duration = event_duration_intervals
        if duration > 0:
            events.append([station, event_start, event_end, total_rainfall, duration, max_value, intercurrence])

# Create DataFrame from events list
events_df = pd.DataFrame(events, columns=['STATION', 'START', 'END', 'TOTAL_RAINFALL', 'DURATION', 'MAX_VALUE', 'INTERCURRENCE'])

# Save to CSV
events_df.to_csv('rainfall_events.csv', index=False)



In [ ]:
#PDF
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load the data
file_path = 'rainfall_events.csv'  # Replace with the path to your CSV file
data = pd.read_csv(file_path)

# Convert START and END to datetime
data['START'] = pd.to_datetime(data['START'])
data['END'] = pd.to_datetime(data['END'])

# Filter for station 706
data = data[data['STATION'] == 706]

# Filter for the time intervals
data_2002_2012 = data[(data['START'].dt.year >= 2002) & (data['START'].dt.year <= 2012)]
data_2013_2023 = data[(data['START'].dt.year >= 2013) & (data['START'].dt.year <= 2023)]

def plot_hist_pdf_side_by_side(data1, data2, column, title, limits_dict, bin_settings,alpha=0.3, log_scale_x=True, log_scale_y=True):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle(f'Catania Normalized Histograms and PDFs of {title}')

    # Apply bin settings if available for the column
    bins = bin_settings.get(column, 'auto')  # default to 'auto' if not specified

    # Apply limits if available for the column
    x_limits = limits_dict.get(column, {}).get('x')
    y_limits = limits_dict.get(column, {}).get('y')

    for i, data in enumerate([data1, data2]):
        sns.histplot(data[column], kde=True, bins=bins, stat="density", linewidth=0, ax=axes[i], alpha=alpha, kde_kws={'bw_adjust': bw_adjust})
        axes[i].set_title(f'{title} {"(2002-2012)" if i == 0 else "(2013-2023)"}')
        axes[i].set_xlabel(title)
        axes[i].set_ylabel('Density')
        if x_limits:
            axes[i].set_xlim(x_limits)
        if y_limits:
            axes[i].set_ylim(y_limits)
        if log_scale_x:
            axes[i].set_xscale('log')
        if log_scale_y:
            axes[i].set_yscale('log')
    plt.tight_layout()
    plt.show()

bw_adjust = 0.5
# Define the limits for each variable
limits = {
    'DURATION': {'x': [10, 300], 'y': [0.0001, 0.1]},
    'TOTAL_RAINFALL': {'x': [1, 200], 'y': [0.0001, 1]},
    'MAX_VALUE': {'x': [1, 31], 'y': [0.0001, 1]},
    'INTERCURRENCE': {'x': [10, 150000], 'y': [0.00000001, 0.001]}
}
# Define the bin settings for each variable
bin_settings = {
    'DURATION': 170,         # Specify number of bins
    'TOTAL_RAINFALL': 150,
    'MAX_VALUE': 70,
    'INTERCURRENCE': 300
}
alpha = 0.3

# Plotting
columns = ['DURATION', 'TOTAL_RAINFALL', 'MAX_VALUE', 'INTERCURRENCE']
for column in columns:
    plot_hist_pdf_side_by_side(data_2002_2012, data_2013_2023, column, column, limits, bin_settings, alpha, log_scale_x=True, log_scale_y=True)
   


In [ ]:
#PDF per city
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
file_path = 'rainfall_events.csv'  # Replace with the path to your CSV file
data = pd.read_csv(file_path)

# Convert START and END to datetime
data['START'] = pd.to_datetime(data['START'])
data['END'] = pd.to_datetime(data['END'])

# Filter for station 706
data = data[data['STATION'] == 706]

# Filter for the time intervals
data_2002_2012 = data[(data['START'].dt.year >= 2002) & (data['START'].dt.year <= 2012)]
data_2013_2023 = data[(data['START'].dt.year >= 2013) & (data['START'].dt.year <= 2023)]

def plot_hist_pdf_side_by_side(data1, data2, column, title, limits_dict, bin_settings, alpha=0.3, log_scale_x=True, log_scale_y=True, save_file=None):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle(f'Catania Normalized Histograms and PDFs of {title}')

    # Apply bin settings if available for the column
    bins = bin_settings.get(column, 'auto')  # default to 'auto' if not specified

    # Apply limits if available for the column
    x_limits = limits_dict.get(column, {}).get('x')
    y_limits = limits_dict.get(column, {}).get('y')

    for i, data in enumerate([data1, data2]):
        sns.histplot(data[column], kde=True, bins=bins, stat="density", linewidth=0, ax=axes[i], alpha=alpha, kde_kws={'bw_adjust': bw_adjust})
        axes[i].set_title(f'{title} {"(2002-2012)" if i == 0 else "(2013-2023)"}')
        axes[i].set_xlabel(title)
        axes[i].set_ylabel('Density')
        if x_limits:
            axes[i].set_xlim(x_limits)
        if y_limits:
            axes[i].set_ylim(y_limits)
        if log_scale_x:
            axes[i].set_xscale('log')
        if log_scale_y:
            axes[i].set_yscale('log')
    plt.tight_layout()
    if save_file:
        plt.savefig(save_file, dpi=300)
    plt.show()

bw_adjust = 0.5
# Define the limits for each variable
limits = {
    'DURATION': {'x': [10, 300], 'y': [0.0001, 0.1]},
    'TOTAL_RAINFALL': {'x': [1, 200], 'y': [0.0001, 1]},
    'MAX_VALUE': {'x': [1, 31], 'y': [0.0001, 1]},
    'INTERCURRENCE': {'x': [10, 150000], 'y': [0.00000001, 0.001]}
}
# Define the bin settings for each variable
bin_settings = {
    'DURATION': 170,         # Specify number of bins
    'TOTAL_RAINFALL': 150,
    'MAX_VALUE': 70,
    'INTERCURRENCE': 300
}
alpha = 0.3

# Plotting and saving
columns = ['DURATION', 'TOTAL_RAINFALL', 'MAX_VALUE', 'INTERCURRENCE']
for column in columns:
    save_file = f"{column}_ct.jpg"
    plot_hist_pdf_side_by_side(data_2002_2012, data_2013_2023, column, column, limits, bin_settings, alpha, log_scale_x=True, log_scale_y=True, save_file=save_file)


In [ ]:
# PDF comparison between stations

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
file_path = 'rainfall_events.csv'  # Replace with the path to your CSV file
data = pd.read_csv(file_path)

# Convert START and END to datetime
data['START'] = pd.to_datetime(data['START'])
data['END'] = pd.to_datetime(data['END'])

# Filter for the time intervals
data_2002_2012 = data[(data['START'].dt.year >= 2002) & (data['START'].dt.year <= 2012)]
data_2013_2023 = data[(data['START'].dt.year >= 2013) & (data['START'].dt.year <= 2023)]

# Combine the two datasets for ease of processing
combined_data = pd.concat([data_2002_2012, data_2013_2023])

def plot_hist_pdf_for_stations(data_2002_2012, data_2013_2023, column, title, limits_dict, bin_settings, station_colors, alpha=0.5, log_scale_x=True, log_scale_y=True, save_file=None):
    fig, axes = plt.subplots(1, 2, figsize=(30, 9))  # Two subplots for each time interval
    fig.suptitle(f'Rainfall Analysis: {title}')

    bins = bin_settings.get(column, 'auto')
    x_limits = limits_dict.get(column, {}).get('x')
    y_limits = limits_dict.get(column, {}).get('y')

    for i, data in enumerate([data_2002_2012, data_2013_2023]):
        ax = axes[i]
        ax.set_title(f'{column} {"(2002-2012)" if i == 0 else "(2013-2023)"}')

        for station, color in station_colors.items():
            station_data = data[data['STATION'] == station]
            sns.kdeplot(station_data[column], ax=ax, color=color, label=f'Station {station}', bw_adjust=bw_adjust)

        ax.set_xlabel(column)
        ax.set_ylabel('Density')
        if x_limits:
            ax.set_xlim(x_limits)
        if y_limits:
            ax.set_ylim(y_limits)
        if log_scale_x:
            ax.set_xscale('log')
        if log_scale_y:
            ax.set_yscale('log')

    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right', title='Stations')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    if save_file:
        plt.savefig(save_file, dpi=300)
    plt.show()

bw_adjust = 0.7

# Define the limits for each variable
limits = {
    'DURATION': {'x': [10, 1000], 'y': [0.0001, 1]},
    'TOTAL_RAINFALL': {'x': [1, 1000], 'y': [0.0001, 1]},
    'MAX_VALUE': {'x': [1, 31], 'y': [0.0001, 1]},
    'INTERCURRENCE': {'x': [10, 10000], 'y': [0.0001, 0.01]}
}
# Define the bin settings for each variable
bin_settings = {
    'DURATION': 170,         # Specify number of bins
    'TOTAL_RAINFALL': 150,
    'MAX_VALUE': 70,
    'INTERCURRENCE': 300
}

alpha = 0.5

station_colors = { # Assuming you have 7 stations
    729: 'blue',
    764: '#9ABDDC',
    706: '#B4CF68',
    756: '#FFD872',
    718: 'orange',
    695: '#FF96C5',
    684: '#FF00FF',
    750: 'purple',
    789: 'black'
}

# Plotting for each variable
columns = ['DURATION', 'TOTAL_RAINFALL', 'MAX_VALUE', 'INTERCURRENCE']
for column in columns:
    save_file = f"rainfall_{column}_stations.jpg"
    plot_hist_pdf_for_stations(data_2002_2012, data_2013_2023, column, column, limits, bin_settings, 
                               station_colors, alpha, log_scale_x=True, log_scale_y=True, save_file=save_file)
    

In [ ]:
# PDF decades
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
file_path = 'rainfall_events.csv'  # Replace with the path to your CSV file
data = pd.read_csv(file_path)

# Convert START and END to datetime
data['START'] = pd.to_datetime(data['START'])
data['END'] = pd.to_datetime(data['END'])

# Filter data for the specified time intervals
data_2002_2012 = data[(data['START'].dt.year >= 2002) & (data['START'].dt.year <= 2012)]
data_2013_2023 = data[(data['START'].dt.year >= 2013) & (data['START'].dt.year <= 2023)]

# Function to plot KDE plots for rainfall data comparison
def plot_kde(data_2002_2012, data_2013_2023, column, title, bw_adjust_2002_2012, bw_adjust_2013_2023, y_limits=None, log_scale_x=True, log_scale_y=True, save_file=None):
    plt.figure(figsize=(12, 6))
    sns.kdeplot(data_2002_2012[column], color='blue', label='2002-2012', bw_adjust=bw_adjust_2002_2012)
    sns.kdeplot(data_2013_2023[column], color='red', label='2013-2023', bw_adjust=bw_adjust_2013_2023)

    plt.title(title)
    plt.xlabel(column)
    plt.ylabel('Density')
    if y_limits:
        plt.ylim(y_limits)
    if log_scale_x:
        plt.xscale('log')
    if log_scale_y:
        plt.yscale('log')

    plt.legend()
    plt.tight_layout()

    if save_file:
        plt.savefig(save_file, dpi=300)
    plt.show()

# Plotting parameters
columns = ['DURATION', 'TOTAL_RAINFALL', 'MAX_VALUE', 'INTERCURRENCE']
bandwidths_2002_2012 = {
    'DURATION': 130,  # Adjust these values as needed for smoothing
    'TOTAL_RAINFALL': 130,
    'MAX_VALUE': 13,
    'INTERCURRENCE': 13
}
bandwidths_2013_2023 = {
    'DURATION': 130,
    'TOTAL_RAINFALL': 130,
    'MAX_VALUE': 13,
    'INTERCURRENCE': 13
}

y_limits = {
    'DURATION': (0.00000001, 1),
    'TOTAL_RAINFALL': (0.00000001, 1),
    'MAX_VALUE': (0.00000001, 1),
    'INTERCURRENCE': (0.00000001, 0.0001)
}

# Plotting for each variable
for column in columns:
    save_file = f"rainfall_{column}_comparison.jpg"
    plot_kde(data_2002_2012, data_2013_2023, column, column, 
             bw_adjust_2002_2012=bandwidths_2002_2012[column], 
             bw_adjust_2013_2023=bandwidths_2013_2023[column], 
             y_limits=y_limits[column], 
             log_scale_x=True, log_scale_y=True, 
             save_file=save_file)

In [ ]:
# increments comparison between stations
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
file_path = 'rainfall_events.csv'  # Replace with the path to your CSV file
data = pd.read_csv(file_path)

# Convert START and END to datetime
data['START'] = pd.to_datetime(data['START'])
data['END'] = pd.to_datetime(data['END'])

# Define the columns for which you want to calculate simple returns
columns = ['DURATION', 'TOTAL_RAINFALL', 'MAX_VALUE', 'INTERCURRENCE']

def calculate_normalized_simple_returns(data, column_name):
    # Calculate simple returns
    data['SIMPLE_RETURNS'] = data[column_name].diff()
    # Normalize simple returns
    mean = data['SIMPLE_RETURNS'].mean()
    std_dev = data['SIMPLE_RETURNS'].std()
    data['NORMALIZED_SIMPLE_RETURNS'] = (data['SIMPLE_RETURNS'] - mean) / std_dev
    return data

# Plotting function for normalized simple returns with explicit axis limits
def plot_normalized_simple_returns(data, column_name, station_colors, x_limits=None, y_limits=None, save_file=None):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle(f'Normalized Simple Returns of {column_name}')

    for time_period, ax in zip(['2002-2012', '2013-2023'], axes):
        time_data = data[(data['START'].dt.year >= int(time_period.split('-')[0])) & (data['START'].dt.year <= int(time_period.split('-')[1]))]
        ax.set_title(f'{column_name} ({time_period})')
        ax.set_xlabel('Normalized Simple Returns')
        ax.set_ylabel('Density')
        ax.set_yscale('log')

        for station in time_data['STATION'].unique():
            station_data = calculate_normalized_simple_returns(time_data[time_data['STATION'] == station].copy(), column_name)
            sns.kdeplot(station_data['NORMALIZED_SIMPLE_RETURNS'].dropna(), ax=ax, color=station_colors.get(station, 'grey'), label=f'Station {station}', bw_adjust=0.5)
        
        # Explicitly set x and y limits if provided
        if x_limits:
            ax.set_xlim(x_limits)
        if y_limits:
            ax.set_ylim(y_limits)
        
        ax.legend()

    plt.tight_layout()
    if save_file:
        plt.savefig(save_file, dpi=300)
    plt.show()

bw_adjust = 31 

# Define x-axis limits for specific columns
x_limits = {
    'DURATION': [-8, 8],
    'TOTAL_RAINFALL': [-15, 15],
    'MAX_VALUE': [-10, 10], 
    'INTERCURRENCE': [-15, 15]
}

# Define y-axis limits for specific columns
y_limits = {
    'DURATION': [0.001, 3],
    'TOTAL_RAINFALL': [0.001, 3],
    'MAX_VALUE': [0.001, 3], 
    'INTERCURRENCE': [0.001, 3]
}

# Define station colors
station_colors = { 
    729: 'blue',
    764: '#9ABDDC',
    706: '#B4CF68',
    756: '#FFD872',
    718: 'orange',
    695: '#FF96C5',
    684: '#FF00FF',
    750: 'purple',
    789: 'black'
}

# Loop through columns and plot
for column in columns:
    save_file = f"{column}_returns.jpg"
    plot_normalized_simple_returns(data, column, station_colors, x_limits.get(column), y_limits.get(column), save_file)


In [ ]:
#fit PDF

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Power-law function
def power_law(x, a, b):
    return a * np.power(x, -b)

# Function to plot the fitted power-law curve
def plot_fit(ax, x, y, a, b, color):
    x_fit = np.logspace(np.log10(np.min(x)), np.log10(np.max(x)), 100)
    y_fit = power_law(x_fit, a, b)
    ax.plot(x_fit, y_fit, color=color, linewidth=1, linestyle='-',  label=f'Power law b=-{b:.2f}')

# Load the data
file_path = 'rainfall_events.csv'
data = pd.read_csv(file_path)

# Convert START and END to datetime and apply other filters
data['START'] = pd.to_datetime(data['START'])
data['END'] = pd.to_datetime(data['END'])
data = data[data['STATION'] != 729]
columns_to_filter = ['TOTAL_RAINFALL', 'DURATION', 'MAX_VALUE', 'INTERCURRENCE']
data = data[(data[columns_to_filter] > 0.1).all(axis=1)]

#data_2013_2023 = data[(data['START'].dt.year >= 2013) & (data['START'].dt.year <= 2023)]
data_2013_2023 = data[(data['START'].dt.year >= 2013) & (data['START'].dt.year <= 2023) & 
                      (data['START'].dt.month.isin([9, 10, 11]))]

# Setting bins for DURATION using logspace
min_duration = data_2013_2023['MAX_VALUE'].min()
max_duration = data_2013_2023['MAX_VALUE'].max()
log_bins_duration = np.logspace(np.log10(min_duration), np.log10(max_duration), 50)

# Histogram plotting
fig, ax = plt.subplots(figsize=(8, 6))

# Calculate histogram with log bins
values, base = np.histogram(data_2013_2023['MAX_VALUE'], bins=log_bins_duration, density=True)
counts, _ = np.histogram(data_2013_2023['MAX_VALUE'], bins=log_bins_duration)

# Calculate bin centers in log space
bin_centers = 10**(0.5 * (np.log10(base[:-1]) + np.log10(base[1:])))

# Filter bins where count > 50
filtered_centers = bin_centers[counts > 100]
filtered_values = values[counts > 100]

# Plot filtered bins
ax.plot(filtered_centers, filtered_values, color='mediumseagreen', marker='o', markersize=7, linestyle='', label='Max per event (2013-2023)')

ax.set_xlabel('Max per event (mm)',fontsize='large')
ax.set_ylabel('Probability Density ',fontsize='large')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(0,4)
#ax.set_ylim(0.01,11)
ax.grid(False)

# Fit to power-law and plot
p0_2013_2023 = [.1, -2]  # Example initial parameters, adjust as needed
params, _ = curve_fit(power_law, filtered_centers, filtered_values, p0=p0_2013_2023)
plot_fit(ax, filtered_centers, filtered_values, params[0], params[1], 'red')

ax.legend(fontsize='x-large')
plt.tight_layout()
plt.savefig('4.fithistogram_max_2013_2023.jpg', dpi=300)
plt.show()

In [ ]:
# fti decumulative probability

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import expi

# Q-Exponential function
def q_exponential(x, A, q, beta):
    return A * np.exp(-beta * x ** q) / expi(q)

# Function to plot the fitted Q-exponential curve
def plot_fit(ax, x, y, A, q, beta, color):
    x_fit = np.logspace(np.log10(np.min(x)), np.log10(np.max(x)), 100)

    y_fit = q_exponential(x_fit, A, q, beta)
    ax.plot(x_fit, y_fit, color=color, linewidth=1, linestyle='-', label=f'Q-Exponential q={q:.2f}')

# Load the data
file_path = 'rainfall_events.csv'
data = pd.read_csv(file_path)

# Convert START and END to datetime and apply other filters
data['START'] = pd.to_datetime(data['START'])
data['END'] = pd.to_datetime(data['END'])
data = data[data['STATION'] != 729]
columns_to_filter = ['TOTAL_RAINFALL', 'DURATION', 'MAX_VALUE', 'INTERCURRENCE']
data = data[(data[columns_to_filter] > 0.1).all(axis=1)]

#data_2013_2023 = data[(data['START'].dt.year >= 2013) & (data['START'].dt.year <= 2023)]
data_2013_2023 = data[(data['START'].dt.year >= 2013) & (data['START'].dt.year <= 2023) & 
                      (data['START'].dt.month.isin([9, 10, 11]))]


# Setting bins for DURATION using linear space
min_duration = data_2013_2023['DURATION'].min()
max_duration = data_2013_2023['DURATION'].max()
#bins_duration = np.logspace(np.log10(min_duration), np.log10(max_duration), 30)
bins_duration = 300
# Histogram plotting
fig, ax = plt.subplots(figsize=(8, 6))

# Calculate histogram with linear bins
values, bins = np.histogram(data_2013_2023['DURATION'], bins=bins_duration, density=True)

# Calculate cumulative probabilities for the x-axis
cdf_values = np.cumsum(values)

# Calculate decumulative probabilities
decumulative_values = 1 - cdf_values / np.sum(values)

# Plot histogram
ax.plot(bins[:-1], decumulative_values, color='mediumseagreen', marker='o', markersize=7, linestyle='', 
        label='Duration event (2013-2023)')

ax.set_xlabel('Duration event Autumn')
ax.set_ylabel('Decumulative Probability')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(0.00011,10)
ax.grid(False)

# Fit to Q-exponential and plot
p0_2013_2023 = [0.09, 1.1, 0.09]  # Example initial parameters, adjust as needed
params, _ = curve_fit(q_exponential, bins[:-1], decumulative_values, p0=p0_2013_2023, maxfev=80000)
plot_fit(ax, bins[:-1], decumulative_values, params[0], params[1], params[2], 'red')

ax.legend()
plt.tight_layout()
plt.savefig('4.fitdec_duration_2013_2023.jpg', dpi=300)
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define the Tsallis Q-Exponential function
def q_exponential(x, A, q, beta):
    return A * (1 - (1 - q) * beta * x) ** (1 / (1 - q))

# Load data from a file. Assume the first row contains column names.
data = pd.read_csv('rainfall_events.csv', header=0, parse_dates=['START', 'END'])

# Filter data to include only events from December, January, and February between 2002 and 2012
filtered_data = data[(data['START'].dt.month.isin([12, 1, 2])) & 
                     (data['START'].dt.year >= 2002) & 
                     (data['START'].dt.year <= 2012)]

# Accessing the "TOTAL_RAINFALL" column from the filtered data
values = filtered_data['TOTAL_RAINFALL'].values

# Define the range of threshold values
thresholds = np.arange(.1, 800.1, .1)

# Calculate the fraction of values greater than each threshold
fractions = [(values > threshold).mean() for threshold in thresholds]

# Parameters for the Tsallis Q-Exponential function (example values, adjust as needed)
A = 1 # Scale factor
q = 1.73 # Entropy index
beta = 2.4 # Scale parameter

# Calculate Tsallis Q-Exponential values for the thresholds
tsallis_values = q_exponential(thresholds, A, q, beta)

# Plot the graph
plt.figure(figsize=(10, 6))
plt.plot(thresholds, fractions, 'o', markersize=4, label='Depth (2002-2012)', color='lightskyblue')  # Added 'b^' for blue triangles
plt.plot(thresholds, tsallis_values, label=f'q={q:.2f} k={beta:.2f}', linestyle='--',linewidth=.8, color='red')
plt.xlabel('Depth (mm)',fontsize='large')
plt.ylabel('Decumuative probability',fontsize='large')
plt.ylim(0.01, 2)
plt.xlim(0.1,100)
plt.xscale('log')  # Using logarithmic scale for x-axis
plt.yscale('log')  # Using logarithmic scale for y-axis
plt.legend(fontsize='x-large')
#plt.tick_params(axis='both', which='major', labelsize=16)
plt.grid(False) 
plt.savefig('1.fit_decumulative_depth_2002_2012.jpg', dpi=300)
plt.show()

In [ ]:
# fit frequency vs ranking 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data
file_path = 'rainfall_events.csv'
data = pd.read_csv(file_path)

# Convert START and END to datetime and apply other filters
data['START'] = pd.to_datetime(data['START'])
data['END'] = pd.to_datetime(data['END'])
data = data[data['STATION'] != 729]
columns_to_filter = ['TOTAL_RAINFALL', 'DURATION', 'MAX_VALUE', 'INTERCURRENCE']
data = data[(data[columns_to_filter] > 0.1).all(axis=1)]

#data_2013_2023 = data[(data['START'].dt.year >= 2002) & (data['START'].dt.year <= 2012)]
data_2013_2023 = data[(data['START'].dt.year >= 2013) & (data['START'].dt.year <= 2023) & 
                      (data['START'].dt.month.isin([3,4, 5]))]


# Rank the durations by frequency and normalize the counts
duration_counts = data_2013_2023['INTERCURRENCE'].value_counts(normalize=True)
duration_ranks = np.arange(1, len(duration_counts) + 1)

# Plot frequency against rank
plt.figure(figsize=(8, 6))
plt.loglog(duration_ranks, duration_counts.values, marker='o', linestyle='', color='blue')
plt.xlabel('Rank')
plt.ylabel('Frequency (Normalized)')
plt.title('Zipf\'s Law - Duration')
plt.grid(True)

# Fit a line to the data
slope, intercept = np.polyfit(np.log(duration_ranks), np.log(duration_counts.values), 1)
plt.plot(duration_ranks, np.exp(intercept) * np.power(duration_ranks, slope), color='red', label=f'Fit (slope={slope:.2f})')
plt.legend()

plt.tight_layout()
#plt.savefig('zipf_duration_fit_normalized.jpg', dpi=300)
plt.show()



In [ ]:
# fit increments

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Define the Tsallis Q-Gaussian function
def q_gaussian(x, A, q, beta, mu):
    term = 1 - (1 - q) * beta * (x - mu)**2
    valid = term > 0
    result = np.zeros_like(x)
    if valid.any():
        result[valid] = A * np.power(term[valid], 1 / (1 - q))
    return result

# Load and preprocess data
file_path = 'rainfall_events.csv'
data = pd.read_csv(file_path)
data['START'] = pd.to_datetime(data['START'])
data['END'] = pd.to_datetime(data['END'])

# Filter data by years (2002 to 2012) and remove station 729
#data = data[(data['START'].dt.year >= 2002) & (data['START'].dt.year <= 2012)]
data = data[(data['START'].dt.year >= 2013) & (data['START'].dt.year <= 2023) & 
            (data['START'].dt.month.isin([6, 7, 8]))]
#data = data[data['STATION'] != 729]

# Function to calculate normalized simple returns
def calculate_normalized_simple_returns(data, column_name):
    data['SIMPLE_RETURNS'] = data[column_name].diff()
    mean = data['SIMPLE_RETURNS'].mean()
    std_dev = data['SIMPLE_RETURNS'].std()
    data['NORMALIZED_SIMPLE_RETURNS'] = (data['SIMPLE_RETURNS'] - mean) / std_dev
    return data

# Combine data from all stations
combined_data = pd.concat([data[data['STATION'] == station] for station in data['STATION'].unique()])
combined_data = calculate_normalized_simple_returns(combined_data, 'INTERCURRENCE')

# Parameters for the Tsallis Q-Gaussian function (example values, adjust as needed)
A = 2.2  # Amplitude
q = 2.45
beta = 2500 # Scale parameter
mu = 0  # Mean

# Generate values for the Tsallis Q-Gaussian
x_values = np.linspace(-13, 13, 400)
y_values = q_gaussian(x_values, A, q, beta, mu)

# Plotting 250bins
fig, ax = plt.subplots(figsize=(10, 5))
#fig.suptitle('Max Simple Returns')
sns.histplot(combined_data['NORMALIZED_SIMPLE_RETURNS'].dropna(), bins=200, 
             kde=False, ax=ax, color='mediumseagreen', label='Interevents (2013-2023)')
ax.plot(x_values, y_values * len(combined_data['NORMALIZED_SIMPLE_RETURNS'].dropna()) * (16 / 30), color='red', label=f'q={q:.2f}, β={beta:.2f}')  # Scale curveax.set_xlabel('Normalized Simple Returns', fontsize='large')
ax.set_ylabel('Probability Density', fontsize='large')
ax.set_xlabel('Normalized simple returns', fontsize='large')
ax.set_yscale('log')
ax.set_xlim(-4,4)
ax.set_ylim(0.9, 50000)
ax.legend(fontsize='x-large')  # Enlarged legend
plt.tight_layout()
plt.savefig('3.returns_inter_2013_2023.jpg', dpi=300)
plt.show()